In [2]:
import importlib
from osgeo import gdal
import pprint
import json
import numpy as np
import matplotlib.pyplot as plt
import itertools
from collections import Counter

# 0 => clear land pixel
# 1 => clear water pixel
# 2 => cloud shadow
# 3 => snow
# 4 => cloud

In [3]:
import os

directory = f"/Volumes/X/Data/fusion-s1-s2/s2/fmask/"  # replace with the path to your directory
fmask_paths = [
    f"/Volumes/X/Data/fusion-s1-s2/s2/fmask/" +
    folder_name +
    "/" +
    folder_name +
    "_fmask.tif"
    for folder_name in os.listdir(directory)
]

In [4]:
MAP_CLOUD = {
    0: "clear_land",
    1: "cloud_water", 
    2: "cloud_shadow",
    3: "snow",
    4: "cloud",
    255: "no_observation"
}

In [18]:
fmim= gdal.Open("/Volumes/X/Data/fusion-s1-s2/s2/fmask/S2_32VNH_20201120/S2_32VNH_20201120_fmask.tif")
im = gdal.Open("/Volumes/X/Data/fusion-s1-s2/s2/sre-10m/S2_32VNH_20210822/S2_32VNH_20210822_B02.tif")

In [40]:
width = im.RasterXSize
height = im.RasterYSize

gt = im.GetGeoTransform()

minx = gt[0]
miny = gt[3]
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3] 

print(gt[0])  
print(width, gt[1], gt[0] + width*gt[1] - minx)
print(height, gt[3], gt[3] + height*gt[5] - miny)
print(gt[3])

499980.0
10980 10.0 109800.0
10980 6300000.0 -109800.0
6300000.0


In [5]:
json_data = {}

for fmask_path in fmask_paths:
    try:
        gdal_data = gdal.Open(fmask_path)
        data = gdal_data.ReadAsArray()
        print(data.shape)
    except:
        print("File not exist here: ", fmask_path)
        vrt.tif_to_vrt("/".join(fmask_path.split("/")[:-1]) + "/fmask.tif", vrt_path)
        gdal_data = gdal.Open(vrt_path)
        data = gdal_data.ReadAsArray()
    
    shape = data.size
    
    count = dict(Counter(itertools.chain(*data)))
    map = {MAP_CLOUD[k]: round((vshape)*100, 2) for k, v in count.items()}
    
    json_data[fmask_path.split("/")[-1]] = dict(sorted(map.items(), key=lambda x: -x[1]))

with open("fmask_stats.json", "w") as f:
    json.dump(json_data, f)

(5490, 5490)
(5490, 5490)
(5490, 5490)
(5490, 5490)


KeyboardInterrupt: 